## Analyzing and visualizing proteomics dataset from shotgun and phospho proteome of drug tolerant persisters

Conditions: 6 cell line (2 KRAS: H358, H23; 2 EGFR: PC9, H1975, ALK: H2228, H3122), 3 replicates (TMT-10plex), 3 time-points (day 0, day2 and day9).
The data has been analyzed for all 6 globally together; now separating them into genotypes (KRAS: parental vs persister comparison)

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import scipy.stats as stats
import statsmodels.api as sm
import scipy.stats as stats
%matplotlib inline


In [ ]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

In [ ]:
# read the files
H358_proteins = pd.read_excel('H358.xlsx')
H23_proteins = pd.read_excel('H23.xlsx')
H1975_proteins = pd.read_excel('H1975.xlsx')
PC9_proteins = pd.read_excel('PC9.xlsx')
H3122_proteins = pd.read_excel('H3122.xlsx')
H2228_proteins = pd.read_excel('H2228.xlsx')


print(H358_proteins.head(20))
print(H23_proteins.head(20))
print (H1975_proteins.head(20))
print (PC9_proteins.head(20))
print (H3122_proteins.head(20))
print (H2228_proteins.head(20))


In [ ]:
#Make copy of the dataframes
H358_proteins_copy = H358_proteins.copy()
H23_proteins_copy = H23_proteins.copy()
H1975_proteins_copy = H1975_proteins.copy()
PC9_proteins_copy = PC9_proteins.copy()
H3122_proteins_copy = H3122_proteins.copy()
H2228_proteins_copy = H2228_proteins.copy()

In [ ]:
# Customize the regular expression pattern to extract gene names without _HUMAN
pattern = r'\|([A-Z0-9_]+)\|([A-Z0-9_]+)_HUMAN'
# Define a function to extract gene names from the string and remove _HUMAN
def extract_gene_names(input_string):

    gene_names = re.findall(pattern, input_string)
    if gene_names:
        return [name[1] for name in gene_names]
    else:
        return None

In [ ]:
# Apply the regex to the column names


H358_proteins['Protein_name'] = H358_proteins['Protein'].apply(extract_gene_names)
H23_proteins['Protein_name'] = H23_proteins['Protein'].apply(extract_gene_names)
H1975_proteins['Protein_name'] = H1975_proteins['Protein'].apply(extract_gene_names)
PC9_proteins['Protein_name'] = PC9_proteins['Protein'].apply(extract_gene_names)
H3122_proteins['Protein_name'] = H3122_proteins['Protein'].apply(extract_gene_names)
H2228_proteins['Protein_name'] = H2228_proteins['Protein'].apply(extract_gene_names)


In [ ]:
H358_proteins.head(20)

In [ ]:
H358_proteins_x = H358_proteins.explode('Protein_name')
H358_proteins_x.head(15)

In [ ]:
H23_proteins_x = H23_proteins.explode('Protein_name')
H23_proteins_x.head(40)

In [ ]:
#Other cell lines explode
H1975_proteins_x = H1975_proteins.explode('Protein_name')
PC9_proteins_x = PC9_proteins.explode('Protein_name')
H3122_proteins_x = H3122_proteins.explode('Protein_name')
H2228_proteins_x = H2228_proteins.explode('Protein_name')

In [ ]:
#H358_proteins_x.to_excel('H358_proteins_x.xlsx')
#H23_proteins_x.to_excel('H23_proteins_x.xlsx')
H1975_proteins_x.to_excel('H1975_proteins_x.xlsx')
PC9_proteins_x.to_excel('PC9_proteins_x.xlsx')
H3122_proteins_x.to_excel('H3122_proteins_x.xlsx')
H2228_proteins_x.to_excel('H2228_proteins_x.xlsx')

In [ ]:
H358_proteins_x["Condition"].unique()
H23_proteins_x["Condition"].unique()
H1975_proteins_x["Condition"].unique()
PC9_proteins_x["Condition"].unique()
H3122_proteins_x["Condition"].unique()
H2228_proteins_x["Condition"].unique()

Now we have to separate the conditions into different columns: two ways to do it- pivot table and one hot encoding. Examples below: import pandas as pd

##### Pivot table: generating ones

###### Sample DataFrame
data = {'Unique_Column': ['Value1', 'Value2', 'Value3', 'Value1', 'Value3']}
df = pd.DataFrame(data)

###### Use groupby to generate 3 new columns based on the unique values
grouped_df = df.groupby(df.index)['Unique_Column'].value_counts().unstack(fill_value=0)

print(grouped_df)

##### Using groupby

###### Sample DataFrame
data = {'Unique_Column': ['Value1', 'Value2', 'Value3', 'Value1', 'Value3']}
df = pd.DataFrame(data)

###### Create a new column "Indicator" to indicate the presence of unique values
df['Indicator'] = 1

###### Use pivot_table to generate 3 new columns based on the unique values
pivot_df = df.pivot_table(index=df.index, columns='Unique_Column', values='Indicator', fill_value=0)

print(pivot_df)


##### ONE hot Encoding: generating zeros
import pandas as pd

###### Sample DataFrame
data = {'Unique_Column': ['Value1', 'Value2', 'Value3', 'Value1', 'Value3']}
df = pd.DataFrame(data)

###### Use one hot encoding to create new columns based on unique values
one_hot_encoded = pd.get_dummies(df['Unique_Column'])

###### Concatenate the one hot encoded columns with the original DataFrame
df_encoded = pd.concat([df, one_hot_encoded], axis=1)

###### Drop the original "Unique_Column" as it is no longer needed
df_encoded.drop('Unique_Column', axis=1, inplace=True)

print(df_encoded)


In [ ]:
# Using Pivot table: circumvents the groupby issue
H358_proteins_pivot = H358_proteins_x.pivot_table(index= ['Protein', 'Protein_name'], columns='Condition', values='Abundance', aggfunc='mean')
H23_proteins_pivot = H23_proteins_x.pivot_table(index= ['Protein', 'Protein_name'], columns='Condition', values='Abundance', aggfunc='mean')
H1975_proteins_pivot = H1975_proteins_x.pivot_table(index= ['Protein', 'Protein_name'], columns='Condition', values='Abundance', aggfunc='mean')
PC9_proteins_pivot = PC9_proteins_x.pivot_table(index= ['Protein', 'Protein_name'], columns='Condition', values='Abundance', aggfunc='mean')
H3122_proteins_pivot = H3122_proteins_x.pivot_table(index= ['Protein', 'Protein_name'], columns='Condition', values='Abundance', aggfunc='mean')
H2228_proteins_pivot = H2228_proteins_x.pivot_table(index= ['Protein', 'Protein_name'], columns='Condition', values='Abundance', aggfunc='mean')

In [ ]:
# Merging the two dataframes
merged_df = H358_proteins_pivot.merge(H23_proteins_pivot, on='Protein_name', how='outer') \
           .merge(H1975_proteins_pivot, on='Protein_name', how='outer') \
           .merge(PC9_proteins_pivot, on='Protein_name', how='outer') \
           .merge(H3122_proteins_pivot, on='Protein_name', how='outer') \
           .merge(H2228_proteins_pivot, on='Protein_name', how='outer')

merged_df.head(40)

In [ ]:
# Replacing the NaN values with 0
merged_df = merged_df.fillna(0)
merged_df.head(10)

In [ ]:
# Resetting the index
merged_df_ri = merged_df.reset_index().sort_values(by= "Protein_name")
merged_df_ri.head(10)

In [ ]:
# Hide index
merged_df_ri.style.hide_index() 
merged_df_ri.head(10)

In [ ]:
cols= merged_df_ri.columns[1:] 
cols

In [ ]:
# Let's look at the distribution of the data
cols= merged_df_ri.columns[1:] 
for col in cols:
    sns.kdeplot(merged_df_ri[col], shade=True)

In [ ]:
num_rows = 1
num_cols = len(cols)
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 5))

for i, col in enumerate(cols):
    sns.kdeplot(merged_df_ri[col], shade=True, ax=axes[i])
    axes[i].set_xlabel('X-axis Label')  # Customize the x-axis label (optional)
    axes[i].set_ylabel('Density')  # Customize the y-axis label (optional)
    axes[i].set_title(col)
    
plt.show()

In [ ]:
sns.pairplot(merged_df_ri, diag_kind='kde')

In [ ]:
# pandas profiling
import pandas_profiling
merged_df_ri.profile_report()

#### P-VALUE turned out to be not so meaningful (ERROR in p value calculation; correction below), the dynamic range is very thin; try a median normalization

In [ ]:
# Setting up control
merged_df_ri['Control_aggregate']= merged_df_ri[['DMSO_H358', 'DMSO_H23', 'DMSO_H1975', 'DMSO_PC9', 'DMSO_H3122', 'DMSO_H2228']].mean(axis=1)

# Setting up treatment
merged_df_ri['Treatment_aggregate']= merged_df_ri[['Osi_d2_H358', 'Osi_d9_H358','Osi_d2_H23', 'Osi_d9_H23', 'Osi_d2_H1975', 'Osi_d9_H1975', 'Osi_d2_PC9', 'Osi_d9_PC9', 'Osi_d2_H3122', 'Osi_d9_H3122', 'Osi_d2_H2228', 'Osi_d9_H2228']].mean(axis=1)

In [ ]:
merged_df_ri.head(10)

In [ ]:
# Std deviation
merged_df_ri['Control_std']= merged_df_ri[['DMSO_H358', 'DMSO_H23', 'DMSO_H1975', 'DMSO_PC9', 'DMSO_H3122', 'DMSO_H2228']].std(axis=1)
merged_df_ri['Treatment_std']= merged_df_ri[['Osi_d2_H358', 'Osi_d9_H358','Osi_d2_H23', 'Osi_d9_H23', 'Osi_d2_H1975', 'Osi_d9_H1975', 'Osi_d2_PC9', 'Osi_d9_PC9', 'Osi_d2_H3122', 'Osi_d9_H3122', 'Osi_d2_H2228', 'Osi_d9_H2228']].std(axis=1)

#### Same issue with the p value;

In [ ]:
# manually generating p-value
#Formula: Control_mean - Treated_mean / sqrt((Control_std^2 / n) + (Treated_std^2 / n))
test_stats = []
for gene in merged_df_ri.index:
    test_stat = (merged_df_ri.loc[gene, 'Control_aggregate'] - merged_df_ri.loc[gene, 'Treatment_aggregate']) / np.sqrt((merged_df_ri.loc[gene, 'Control_std']**2 / 2) + (merged_df_ri.loc[gene, 'Treatment_std']**2 / 4))
    test_stats.append(test_stat)

In [ ]:
merged_df_ri['test_stats'] = test_stats
merged_df_ri.head(10)

In [ ]:
# Trying a for loop
df = merged_df_ri

# Separate the treated and untreated cohorts
untreated_samples = ['DMSO_H358', 'DMSO_H23', 'DMSO_H1975', 'DMSO_PC9', 'DMSO_H3122', 'DMSO_H2228']  
treated_samples = ['Osi_d2_H358', 'Osi_d9_H358','Osi_d2_H23', 'Osi_d9_H23', 'Osi_d2_H1975', 'Osi_d9_H1975', 'Osi_d2_PC9', 'Osi_d9_PC9', 'Osi_d2_H3122', 'Osi_d9_H3122', 'Osi_d2_H2228', 'Osi_d9_H2228']  

# Create separate DataFrames for treated and untreated cohorts
treated_df = df[treated_samples]
untreated_df = df[untreated_samples]

# Calculate the mean expression for each gene in treated and untreated cohorts
mean_treated = treated_df.mean(axis=1)
mean_untreated = untreated_df.mean(axis=1)

In [ ]:
mean_treated.head(10)

In [ ]:
mean_untreated.head(10)

In [ ]:
# Calculate LogFC
logFC = np.log2(mean_treated / mean_untreated)

# Calculate the absolute LogFC
abs_logFC = np.abs(logFC)
abs_logFC.head(10)

In [ ]:
df.index

In [ ]:
# Perform a statistical test (e.g., t-test) for each gene
p_values = []
for gene in df.index:
    _, p_value = stats.ttest_ind(treated_df.loc[gene], untreated_df.loc[gene])
    p_values.append(p_value)
    
p_values

In [ ]:
# printing the first few p-values
print (p_values[:10])
print (len(p_values))

In [ ]:
df.head(10)

#### So, the issue is with the multiple hypothesis testing; Let's try to manually do it

In [ ]:
# Total number of genes (N)
total_genes = len(p_values)

# Calculate the ranks of p-values
ranked_p_values = np.argsort(p_values) + 1

# Calculate padj for each gene
padj = [p * (total_genes / rank) for p, rank in zip(p_values, ranked_p_values)]

# Make sure padj values are not greater than 1
padj = [min(p, 1.0) for p in padj]

print (padj)
print (len(padj))
print (min(padj))
print (max(padj))

In [ ]:
df['padj'] = padj
df.head(10)

In [ ]:
df['Significant']= df['padj'] < 0.05
df.head(10)

In [ ]:
# Filter differentially expressed genes based on significance level (e.g., padj < 0.05)
DEG = df[df['Significant'] == True]

In [ ]:
DEG.head(15)

In [ ]:
DEG.shape

In [ ]:
DEG["FDR"] = DEG["padj"].apply(lambda x: -1 * (x if x > 0 else 1e-300)).apply(lambda x: -1 * (x if x < 1 else 1))
DEG.head(10)

In [ ]:
# Check dtype in DEG FDR column
DEG.dtypes

In [ ]:
DEG['-Log10FDR'] = -1* np.log10(DEG['FDR'])

In [ ]:
DEG.head(25)

In [ ]:
# Generate a volcano plot
# Define the threshold for log fold change

logFC_threshold = 0.05

plt.figure(figsize=(10, 6))
plt.scatter(DEG["LogFC"], DEG["-Log10FDR"], c="blue", edgecolors="black", alpha=0.7)

plt.xlabel("Log Fold Change (LogFC)")
plt.ylabel("-log10(False Discovery Rate) FDR")
plt.title("Volcano Plot")

significant_logfc_threshold = 0.05
significant_fdr_threshold = -1 * (1 / 1000)  # Equivalent to FDR = 0.001
plt.axvline(significant_logfc_threshold, color="red", linestyle="--", label="LogFC Threshold")
plt.axhline(significant_fdr_threshold, color="green", linestyle="--", label="-log10(FDR) Threshold")

# Add gene labels
for i, row in DEG.iterrows():
    plt.text(row["LogFC"], row["-Log10FDR"], row["Protein_name"], ha='left', va='bottom')

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
DEG_heatmap = DEG.iloc[:,0:19]
DEG_heatmap.head(10)

In [ ]:
DEG_heatmap.set_index('Protein_name', inplace=True)
DEG_heatmap.head(10) 

In [ ]:
DEG_heatmap.to_excel('DEG_heatmap_all.xlsx')

In [ ]:
# Heatmap of the data
sns.heatmap(DEG_heatmap, cmap="YlGnBu")

In [ ]:
# Clustermap of the data
sns.clustermap(DEG_heatmap, cmap="YlGnBu")

In [ ]:
#Show all the gene names in the heatmap with small font size
sns.set(font_scale=1)
sns.clustermap(DEG_heatmap, cmap="YlGnBu", yticklabels=True, figsize=(50, 50))


In [ ]:
#Importing the data
DEG_heatmap_rc = pd.read_excel('DEG_heatmap_all_organized.xlsx')
#reset the index
DEG_heatmap_rc.set_index('Protein_name', inplace=True)
DEG_heatmap_rc.head(10)

In [ ]:
# Just cluster the rows
sns.clustermap(DEG_heatmap_rc, cmap="YlGnBu", yticklabels=True, figsize=(50, 50), col_cluster=False)